# Algo fraud interoperator

1. Récupère les carpools-identities dans une borne temporelle et géographique à analyser
2. Recherche toutes les collisions pour ces carpools-identities
3. Calcul de statistiques sur les collisions
4. Récupère les incitations sur les drivers pour les trajets collisions et calcul de statistiques

## Récupération des carpools à analyser

In [1]:
import psycopg2
import pandas
import warnings
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

start_date = "2022-11-21 23:59:59"
end_date = "2022-11-23 00:00:01"
aom_insee = "217500016"

con = psycopg2.connect(
    host="localhost",
    database="local",
    user="postgres",
    password="postgres"
)
# CASE WHEN cc.is_driver THEN pa.rpc_driver_uuid ELSE pa.rpc_passenger_uuid END 
query = f"""SELECT cc._id, cc.is_driver, ci.phone_trunc, cc.datetime, cc.duration, cc.operator_id, cc.start_position, cc.end_position, gmap_url(cc.start_position, cc.end_position), pa.rpc_driver_uuid, pa.rpc_passenger_uuid 
FROM CARPOOL.CARPOOLS cc
   join carpool.identities ci on cc.identity_id = ci._id
   join geo.perimeters gps on cc.start_geo_code = gps.arr and gps.year = 2022
   join geo.perimeters gpe on cc.end_geo_code = gpe.arr and gpe.year = 2022
   LEFT join phones.all pa on pa.operator_journey_id = cc.operator_journey_id
WHERE CC.DATETIME >= '{start_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    AND cc.operator_id IN (3,4,9)
	AND CC.DATETIME < '{end_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    {f"and (gps.aom = '{aom_insee}' or gpe.aom = '{aom_insee}') and gps.year = 2022 and gpe.year = 2022" if aom_insee else ""}
"""

df_carpool_iterator = pandas.read_sql(query, con)

total_row_analysed = len(df_carpool_iterator.index)        # nombre de carpool analysés
display(HTML(df_carpool_iterator.head(10).to_html()))      # preview




,_id,is_driver,phone_trunc,datetime,duration,operator_id,start_position,end_position,gmap_url,rpc_driver_uuid,rpc_passenger_uuid
0,21277692,True,+336233586,2022-11-22 06:28:05+00:00,1825,4,0101000020E6100000A87201403A6A004077EFFF9FDE614840,0101000020E6100000742C00207FA201407AF5FF1F1F584840,"https://www.google.com/maps/dir/48.7646064758,2.0518689156/48.6884498596,2.2043440342",513ca4e0-c295-11ed-94f9-024264400007,52f14ef2-c291-11ed-94f9-024264400007
1,21278020,True,+336206605,2022-11-22 06:20:00+00:00,1311,4,0101000020E610000089190120250304406CFDFF7FCD674840,0101000020E61000000D840120770F0540BB070080D7684840,"https://www.google.com/maps/dir/48.8109588623,2.5015356541/48.8190765381,2.6325514317",3f8e8ee8-c295-11ed-94f9-024264400007,16e8871e-c295-11ed-94f9-024264400007
2,21278150,True,+337777896,2022-11-22 08:00:52+00:00,3596,4,0101000020E61000003C550160FC990340F2070060AE644840,0101000020E61000008F4AFE9FFBB501402BEBFF1F15734840,"https://www.google.com/maps/dir/48.7865715027,2.4501883984/48.8990821838,2.2138588428",34635b08-c294-11ed-94f9-024264400007,43b6ace0-c294-11ed-94f9-024264400007
3,21278924,True,+336196017,2022-11-22 08:35:04+00:00,2504,4,0101000020E61000004FF300603D730340A0F7FFDF036C4840,0101000020E610000056AAFEDFD55D0640C6F9FF9F086C4840,"https://www.google.com/maps/dir/48.8438682556,2.4312694073/48.8440132141,2.7958180904",398863a2-c295-11ed-94f9-024264400007,70fff566-c295-11ed-94f9-024264400007
4,21279016,True,+336137274,2022-11-22 15:45:53+00:00,1093,4,0101000020E6100000BF47FF3F5599044064EEFF3F4C4F4840,0101000020E6100000C916FFBFF53A054090FCFFFF91454840,"https://www.google.com/maps/dir/48.6195144653,2.5748696327/48.5435180664,2.6537890434",104238ec-c295-11ed-94f9-024264400007,6e0d0ba2-c293-11ed-94f9-024264400007
5,21280810,True,+337685741,2022-11-22 08:00:23+00:00,1856,4,0101000020E61000003E9200E0D92402403117008046694840,0101000020E61000002565002016580140F4EAFF3FDE604840,"https://www.google.com/maps/dir/48.8224639893,2.2679936886/48.7567825317,2.1680109501",01e8fc8c-c294-11ed-94f9-024264400007,566a9c5a-c291-11ed-94f9-024264400007
6,21273722,True,+336723026,2022-11-22 07:11:25+00:00,1144,4,0101000020E610000086F3004054F5034030F4FFDF15594840,0101000020E6100000AFAFFFBF94BC044043F5FF3F884B4840,"https://www.google.com/maps/dir/48.695980072,2.4947896004/48.59009552,2.5920805931",c9735bba-c291-11ed-94f9-024264400007,21311a14-c291-11ed-94f9-024264400007
7,21273816,True,+336324140,2022-11-22 07:09:38+00:00,2077,4,0101000020E6100000072EEAEE95B4FD3F58000979F3524840,0101000020E610000066C9D23DA15D0140D3FDF73B6F6B4840,"https://www.google.com/maps/dir/48.64805519999999,1.8565883/48.8393321,2.1707177",86414f60-c295-11ed-94f9-024264400007,589e4b2c-c294-11ed-94f9-024264400007
8,21276554,True,+336724042,2022-11-22 16:15:30+00:00,1704,4,0101000020E6100000098CF50D4C4E0640EBAD81AD126C4840,0101000020E6100000D92DB8D5626606403A3BBE62B2844840,"https://www.google.com/maps/dir/48.84432,2.788231/49.0366939,2.7999932",c92a1694-c291-11ed-94f9-024264400007,3b82cb52-c295-11ed-94f9-024264400007
9,21274572,True,+336109114,2022-11-22 07:59:45+00:00,785,4,0101000020E61000000BF2FF1F31F403401FF6FFBFE3694840,0101000020E6100000C4EDFE9FE4DF034072E6FFDF1D6D4840,"https://www.google.com/maps/dir/48.8272628784,2.4942343235/48.8524742126,2.4843227863",fe7fd2a4-c294-11ed-94f9-024264400007,054e1eec-c295-11ed-94f9-024264400007


## Recherche de collisions

- Recherche sur le passager uniquement pour éviter le cas un driver sur plusieurs app mais des passagers différents
- On évite les carpools déjà en collisions pour éviter les doublons
- On doit forcément mettre une zone géographique, même assez large, pour éviter de trouver des collisions sur des `phone_trunc` dans des régions différentes car il est probable que ce soit bien des personnes distinctes

### Notes 

#### Inversion des colonnes passenger /driver uuid sur Karos
Les colonnes `uuid_passager` et `uuid_driver` étaient inversées systématiquement pour Karos.
J'ai pensé au début que le passager et le conducteur inverserai leur role mais l'inversion n'aurait pas été systématique et pas uniquement sur les trajets de Karos.
Je suis parti du principe que c'était une erreur et j'ai donc inverser les colonnes passager et driver pour Karos pour retrouver les mêmes uuids entre opérateurs.
Cela n'empèche pas la detection des collisions en soit mais rend difficile la lecture et l'interprétation des données par la suite.

Requète SQL pour inverser les colonnes de `phones.all` pour Karos 

```sql
UPDATE phones.all 
SET driver = passenger,
    passenger = driver,
	rpc_driver_uuid = rpc_passenger_uuid,
	rpc_passenger_uuid = rpc_driver_uuid
where operator_id = 4;
```

Possible inversion lors de la réception des id Karos via CSV. Il n'y pas le header sur les colonnes du CSV

#### Exemple de recolement par uuid qui ne marche pas 

```sql
SELECT cc._id, pa.rpc_driver_uuid, pa.rpc_passenger_uuid, cc.is_driver, duration, datetime, cc.trip_id, ci.phone_trunc, cc.operator_id
FROM CARPOOL.CARPOOLS cc
JOIN Carpool.identities ci on cc.identity_id = ci._id
JOIN PHONES."all" pa on pa.operator_journey_id = cc.operator_journey_id
WHERE cc.trip_id = '84717f5e-b4a0-485b-92a7-a151bd3c55ed'
	OR cc.trip_id = 'd3ec6a7a-160d-40e3-a733-3dfe8e3a77a6'
```


In [38]:

timeframe = 15                 # timeframe to look at in minue
collisions_tupples = []        # couple de carpool en collisions détectés 
carpools_already_flagged = []  # carpool_id already in a collision hence not checked again with other carpool
geo_frame = 7000               # geo perimeters in metter for start / end in collision detection            

for index, row in df_carpool_iterator.iterrows():
    
# CASE WHEN cc.is_driver THEN pa.rpc_driver_uuid ELSE pa.rpc_passenger_uuid END as rpc_uuid
    df_carpool_collision = pandas.read_sql(f"""
    SELECT cc._id, cc.is_driver, ci.phone_trunc, cc.datetime, cc.duration, cc.operator_id,  gmap_url(cc.start_position, cc.end_position),
    pa.rpc_driver_uuid, pa.rpc_passenger_uuid
    from carpool.carpools cc 
    Join carpool.identities ci on ci._id = cc.identity_id
    LEFT join phones.all pa on pa.operator_journey_id = cc.operator_journey_id
    WHERE cc._id <> {row['_id']}
    AND cc.is_driver = false
    AND cc.operator_id IN (3,4,9)
    {f"AND cc._id not in ({','.join(str(x) for x in carpools_already_flagged)})" if carpools_already_flagged else ""}
    AND CC.DATETIME >= '{start_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
	AND CC.DATETIME < '{end_date}'::timestamp AT TIME ZONE 'EUROPE/PARIS'
    AND ci.phone_trunc = '{row['phone_trunc']}'
    and cc.operator_id <> {row['operator_id']}
    AND '{row['datetime']}'::timestamp >= cc.datetime - '{timeframe} minutes'::interval
            and '{row['datetime']}'::timestamp <= cc.datetime + '{timeframe} minutes'::interval
    AND ((CC.DATETIME < '{row['datetime']}'::timestamp
						AND CC.DATETIME + (CC.DURATION || ' seconds')::interval > '{row['datetime']}'::timestamp)
						OR (CC.DATETIME > '{row['datetime']}'::timestamp
						AND '{row['datetime']}'::timestamp + ({row['duration']} || ' seconds')::interval > CC.DATETIME)
						OR (CC.DATETIME < '{row['datetime']}'
						AND CC.DATETIME + (CC.DURATION  || ' seconds')::interval< '{row['datetime']}'::timestamp + ({row['duration']} || ' seconds')::interval)
						OR (CC.DATETIME > '{row['datetime']}'
						AND CC.DATETIME + (CC.DURATION  || ' seconds')::interval> '{row['datetime']}'::timestamp + ({row['duration']} || ' seconds')::interval))
    and st_distance(cc.start_position, '{row['start_position']}'::Geometry) < {geo_frame} and st_distance(cc.end_position, '{row['end_position']}'::Geometry) < {geo_frame}
    """, con)

    if not df_carpool_collision.empty:
        carpools_already_flagged.append(row._id)
        collisions_tupples = collisions_tupples + list(map(lambda x: [row, x[1]] , df_carpool_collision.iterrows()))

        print("\nCollisions:")
        display(row.to_dict())
        display(HTML(df_carpool_collision.to_html()))
        print("-----------------------------------------------------------------------------------------")

KeyboardInterrupt: 

In [10]:
carpool_collision_confirmed = 0      # collisions confirmeés par l'uuid_rpc 
carpool_collision_uuid_mismatch = 0  # collisions infirmées par l'uuid_rpc
flat_carpool_id_collision_mismatch_list = []

for carpool_tupple in collisions_tupples:
    if carpool_tupple[0].rpc_driver_uuid == carpool_tupple[1].rpc_driver_uuid or carpool_tupple[0].rpc_passenger_uuid == carpool_tupple[1].rpc_driver_uuid \
        or carpool_tupple[0].rpc_passenger_uuid == carpool_tupple[1].rpc_passenger_uuid or carpool_tupple[0].rpc_driver_uuid == carpool_tupple[1].rpc_passenger_uuid:
        carpool_collision_confirmed += 1
    else:
        carpool_collision_uuid_mismatch += 1
        print('Collision mismatch')
        print(carpool_tupple[0])
        print(carpool_tupple[1])
        flat_carpool_id_collision_mismatch_list.append(carpool_tupple[0]._id)
        flat_carpool_id_collision_mismatch_list.append(carpool_tupple[1]._id)

carpool_collision_ratio = carpool_collision_confirmed / total_row_analysed # taux confirmés sur total analysé

Collision mismatch
_id                                                            21181701
is_driver                                                         False
phone_trunc                                                  +336446248
datetime                                      2022-11-22 03:36:58+00:00
duration                                                            711
operator_id                                                           3
start_position        0101000020E610000038F3AB3940B00140B6679604A871...
end_position          0101000020E61000009626A5A0DB8B01406475ABE7A473...
gmap_url              https://www.google.com/maps/dir/48.88794,2.211...
rpc_driver_uuid                    58fd5482-c294-11ed-94f9-024264400007
rpc_passenger_uuid                 260074c2-c291-11ed-94f9-024264400007
Name: 225, dtype: object
_id                                                            21281695
is_driver                                                         False
phone_trunc         

In [36]:
from numpy import isnan


flat_collided_driver_carpool_confirmed = [] # list des drivers et leur carpool id pour les collisions

for tupple in collisions_tupples:
    flat_collided_driver_carpool_confirmed.append(tupple[0])
    flat_collided_driver_carpool_confirmed.append(tupple[1])

flat_collided_driver_carpool_confirmed = list(filter(lambda c: c.is_driver is True and c._id not in flat_carpool_id_collision_mismatch_list, flat_collided_driver_carpool_confirmed))

df_carpool_collided_with_incentives = pandas.read_sql(f"""
        SELECT cc._id, pi.amount, pi.result
        FROM CARPOOL.CARPOOLS cc
        LEFT JOIN policy.incentives pi on pi.carpool_id = cc._id and pi.policy_id = 459
        WHERE cc._ID in ({','.join(str(x._id) for x in flat_collided_driver_carpool_confirmed)});
        """, con)

incentive_sum_frauded = 0
incentive_frauded_count = 0
for index, frauded in df_carpool_collided_with_incentives.iterrows():
    if not isnan(frauded.result) : 
        incentive_frauded_count += 1
        incentive_sum_frauded += frauded.result

## Conclusion

Une collision: Un `phone_trunc` sur 2 trajets et 2 opérateurs différents

Sur IDFM sur le 22 Novembre 2022 (7 km de rayon sur le start et le end)

- Nombre de trajets : 18902
- Nombre de collisions : 999 (5,2%)
- Nombre de collisions confirmés par comparaison des uuids "malette" : 981 (98%)
- Nombre de trajets (operator_journey_id) en collisions: 1515
- Nombre de trajets (operator_journey_id) en collisions confirmés par comparaison des uuids, incités (ou trajets frauduleux): 253 
- Sommes des incitations sur les trajets incités: 527,70€

### Notes
- Le nombre de collisions est beaucoup plus important lorsqu'on inclu les collisions driver - driver => 2234 collisions Cas légal mais fraude possible sur covoit boite à gants

### Améliorations 
1. Determiner le sens d'un trajet afin d'éviter les A/R sur plusieurs opérateurs. Via concaténation de 2 labels / strings représentant l'O et la D
2. Comparer avec les incitations envoyées par les opérateurs. A priori, le delta est faible